In [ ]:
import pandas as pd
from utils.gcp import CloudStorageOps
import io

In [ ]:
PATH_FILE = "raw_data/CIDDS-001-external-week1.csv"

gcs = CloudStorageOps("ml-anomaly-detection")

load = gcs.load_from_bucket(PATH_FILE)

if load is not None:
    df = pd.read_csv(io.BytesIO(load))
else:
    print("Load returned None, cannot read CSV.")

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172838 entries, 0 to 172837
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Date first seen    172838 non-null  object 
 1   Duration           172838 non-null  float64
 2   Proto              172838 non-null  object 
 3   Src IP Addr        172838 non-null  object 
 4   Src Pt             172838 non-null  int64  
 5   Dst IP Addr        172838 non-null  object 
 6   Dst Pt             172838 non-null  float64
 7   Packets            172838 non-null  int64  
 8   Bytes              172838 non-null  object 
 9   Flows              172838 non-null  int64  
 10  Flags              172838 non-null  object 
 11  Tos                172838 non-null  int64  
 12  class              172838 non-null  object 
 13  attackType         172838 non-null  object 
 14  attackID           172838 non-null  object 
 15  attackDescription  172838 non-null  object 
dtypes:

In [21]:
cols_to_rename = {
    "Date first seen" :"session_start_time",
    "Duration": "duration",
    "Proto": "network_protocol",
    "Src IP Addr": "source_ip_addres",
    "Src Pt": "source_port",
    "Dst IP Addr": "dest_ip_address",
    "Dst Pt": "dest_port",
    "Packets": "total_packets_used",
    "Bytes": "bytes_flow",
    "Flows": "flows",
    "Flags": "network_flags",
    "Tos": "tos",
    "class": "class",
    "attackType": "attack_type",
    "attackID": "attack_id",
    "attackDescription": "attack_description",
}

df = df.rename(columns=cols_to_rename)

In [22]:
df.sample(5)

,session_start_time,duration,network_protocol,source_ip_addres,source_port,dest_ip_address,dest_port,total_packets_used,bytes_flow,flows,network_flags,tos,class,attack_type,attack_id,attack_description
94581,2017-03-18 22:16:28.335,30.999,TCP,EXT_SERVER,80,15588_32,37131.0,6,264,1,.A..S.,0,unknown,---,---,---
120179,2017-03-20 06:43:17.727,5.683,TCP,EXT_SERVER,22,10006_27,41489.0,10,1680,1,.AP.SF,0,suspicious,---,---,---
58671,2017-03-17 17:54:09.364,13.301,TCP,EXT_SERVER,22,13976_197,4726.0,18,2985,1,.AP.S.,0,suspicious,---,---,---
167102,2017-03-21 18:36:40.062,0.084,TCP,EXT_SERVER,8000,OPENSTACK_NET,55185.0,7,561,1,.AP.SF,0,normal,---,---,---
166317,2017-03-21 18:05:25.245,17.121,TCP,18851_139,41917,EXT_SERVER,22.0,20,2529,1,.APRSF,0,suspicious,---,---,---


### Dicionário de Dados - CIDDS-001

- **session_start_time**: Timestamp em que a sessão de rede começou.
- **duration**: Duração da sessão (em segundos).
- **network_protocol**: Protocolo de comunicação usado (TCP, UDP, ICMP, etc.).
- **source_ip_addres**: Endereço IP de origem do tráfego.
- **source_port**: Porta de origem do tráfego.
- **dest_ip_address**: Endereço IP de destino do tráfego.
- **dest_port**: Porta de destino do tráfego.
- **total_packets_used**: Número total de pacotes transmitidos na sessão.
- **bytes_flow**: Quantidade total de bytes transferidos.
- **flows**: Quantidade de fluxos (connections) relacionados à sessão.
- **network_flags**: Sinalizadores usados no protocolo (como SYN, ACK, FIN).
- **tos**: Tipo de Serviço (Type of Service) – define prioridade e tratamento do pacote na rede.
- **class**: Rótulo de classificação da conexão, podendo ser 'normal' ou 'attack'.
- **attack_type**: Tipo de ataque detectado (por exemplo: portscan, bruteforce, etc.).
- **attack_id**: Identificador único do ataque.
- **attack_description**: Descrição detalhada do ataque, se aplicável.


In [23]:
df["session_start_time"] = df["session_start_time"].apply(pd.to_datetime)

In [24]:
df["total_packets_used"] = pd.to_numeric(df["total_packets_used"], errors="coerce")
df["bytes_flow"] = pd.to_numeric(df["bytes_flow"], errors="coerce")
df["source_port"] = df["source_port"].astype("float64")

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172838 entries, 0 to 172837
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   session_start_time  172838 non-null  datetime64[ns]
 1   duration            172838 non-null  float64       
 2   network_protocol    172838 non-null  object        
 3   source_ip_addres    172838 non-null  object        
 4   source_port         172838 non-null  float64       
 5   dest_ip_address     172838 non-null  object        
 6   dest_port           172838 non-null  float64       
 7   total_packets_used  172838 non-null  int64         
 8   bytes_flow          172779 non-null  float64       
 9   flows               172838 non-null  int64         
 10  network_flags       172838 non-null  object        
 11  tos                 172838 non-null  int64         
 12  class               172838 non-null  object        
 13  attack_type         172838 no

In [26]:
df.isna().sum()

,0
session_start_time,0
duration,0
network_protocol,0
source_ip_addres,0
source_port,0
dest_ip_address,0
dest_port,0
total_packets_used,0
bytes_flow,59
flows,0


In [27]:
df = df.dropna()

In [72]:
df.isna().sum()

,0
session_start_time,0
duration,0
network_protocol,0
source_ip_addres,0
source_port,0
dest_ip_address,0
dest_port,0
total_packets_used,0
bytes_flow,0
network_flags,0


In [29]:
df["network_protocol"] = df["network_protocol"].str.strip().str.upper()
df["network_flags"] = df["network_flags"].str.strip()
df["class"] = df["class"].str.strip().str.lower()

In [31]:
cols = df.columns.tolist()
cols

['session_start_time',
 'duration',
 'network_protocol',
 'source_ip_addres',
 'source_port',
 'dest_ip_address',
 'dest_port',
 'total_packets_used',
 'bytes_flow',
 'flows',
 'network_flags',
 'tos',
 'class',
 'attack_type',
 'attack_id',
 'attack_description']

In [33]:
df["bytes_per_packet"] = df["bytes_flow"]/df["total_packets_used"]
df["packets_per_seconds"] = df["total_packets_used"]/df["duration"]
df["hour_of_day"] = df["session_start_time"].dt.hour

In [76]:
common_ports = [80.0, 443.0, 22.0, 21.0]
df["is_common_port"] = [1 if port in common_ports else 0 for port in df["dest_port"]]

In [48]:
df["class"].value_counts(normalize=True)

,proportion
class,
suspicious,0.621239
normal,0.286806
unknown,0.091956


In [49]:
df["class"] = df["class"].apply(lambda x: "suspicious" if x in ["suspicious", "unknown"] else x)

In [85]:
with_new_cols = df.columns.tolist()
with_new_cols

['session_start_time',
 'duration',
 'network_protocol',
 'source_ip_addres',
 'source_port',
 'dest_ip_address',
 'dest_port',
 'total_packets_used',
 'bytes_flow',
 'network_flags',
 'bytes_per_packet',
 'packets_per_seconds',
 'hour_of_day',
 'is_common_port',
 'class']

In [84]:
df = df[[
    'session_start_time',
    'duration',
    'network_protocol',
    'source_ip_addres',
    'source_port',
    'dest_ip_address',
    'dest_port',
    'total_packets_used',
    'bytes_flow',
    'network_flags',
    'bytes_per_packet',
    'packets_per_seconds',
    'hour_of_day',
    'is_common_port',
    'class'
]]

In [63]:
df.sample(5)

,session_start_time,duration,network_protocol,source_ip_addres,source_port,dest_ip_address,dest_port,total_packets_used,bytes_flow,network_flags,bytes_per_packet,packets_per_seconds,hour_of_day,is_common_port,class
96349,2017-03-18 23:43:42.285,0.043,TCP,OPENSTACK_NET,56345.0,EXT_SERVER,8000.0,6,614.0,.AP.SF,102.333333,139.534884,23,0,normal
94309,2017-03-18 23:06:40.697,0.071,TCP,EXT_SERVER,8000.0,OPENSTACK_NET,55706.0,7,1723.0,.AP.SF,246.142857,98.591549,23,0,normal
100898,2017-03-19 04:49:43.548,0.048,TCP,EXT_SERVER,8000.0,OPENSTACK_NET,56829.0,7,639.0,.AP.SF,91.285714,145.833333,4,0,normal
128299,2017-03-20 12:44:10.705,0.981,TCP,17173_42,62484.0,EXT_SERVER,22.0,6,340.0,.AP.SF,56.666667,6.116208,12,0,suspicious
113155,2017-03-19 22:34:40.572,0.041,TCP,EXT_SERVER,8000.0,OPENSTACK_NET,63655.0,7,556.0,.AP.SF,79.428571,170.731707,22,0,normal


In [78]:
df["is_common_port"].value_counts(normalize=True)

,proportion
is_common_port,
0,0.718241
1,0.281759


In [81]:
df["network_flags"].value_counts(normalize=True)

,proportion
network_flags,
.AP.SF,0.600316
....S.,0.104324
.APRSF,0.083673
.A.R..,0.063081
.A..S.,0.054914
.AP.S.,0.027671
......,0.023371
.APRS.,0.015384
...RS.,0.011165


In [87]:
df["has_SYN"] = df["network_flags"].str.contains("S").astype(int)
df["has_ACK"] = df["network_flags"].str.contains("A").astype(int)
df["has_RST"] = df["network_flags"].str.contains("R").astype(int)
df["has_FIN"] = df["network_flags"].str.contains("F").astype(int)

In [88]:
df["is_hex_flag"] = df["network_flags"].str.startswith("0x").astype(int)

In [93]:
df.isna().sum()

,0
session_start_time,0
duration,0
network_protocol,0
source_ip_addres,0
source_port,0
dest_ip_address,0
dest_port,0
total_packets_used,0
bytes_flow,0
network_flags,0


In [94]:
import numpy as np

df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

In [95]:
df.sample(3)

,session_start_time,duration,network_protocol,source_ip_addres,source_port,dest_ip_address,dest_port,total_packets_used,bytes_flow,network_flags,bytes_per_packet,packets_per_seconds,hour_of_day,is_common_port,has_SYN,has_ACK,has_RST,has_FIN,is_hex_flag,class
60324,2017-03-17 19:22:45.711,11.728,TCP,10004_36,44577.0,EXT_SERVER,22.0,5,277.0,.APRSF,55.4,0.426330,19,1,1,1,1,1,0,suspicious
118705,2017-03-20 05:25:45.434,0.986,TCP,EXT_SERVER,22.0,17173_42,14193.0,5,1285.0,.AP.SF,257.0,5.070994,5,0,1,1,0,1,0,suspicious
133537,2017-03-20 14:18:21.760,9.024,TCP,10008_109,57596.0,EXT_SERVER,22.0,15,2163.0,.AP.SF,144.2,1.662234,14,1,1,1,0,1,0,suspicious


In [97]:
df["source_ip_addres"]

,source_ip_addres
18,EXT_SERVER
19,10001_123
22,EXT_SERVER
23,10001_123
26,10004_35
...,...
172817,EXT_SERVER
172818,19393_59
172819,EXT_SERVER
172820,18851_139


In [111]:
df["network_protocol"].value_counts()

,count
network_protocol,
TCP,150437
UDP,784
ICMP,111


In [100]:
src_freq = df["source_ip_addres"].value_counts()
df["source_ip_freq"] = df["source_ip_addres"].map(src_freq)

dst_freq = df["dest_ip_address"].value_counts()
df["dest_ip_freq"] = df["dest_ip_address"].map(dst_freq)

In [112]:
df = pd.get_dummies(
    df,
    columns=["network_protocol"],
    dtype=int
)

In [113]:
with_new_cols = df.columns.tolist()
with_new_cols

['session_start_time',
 'duration',
 'source_ip_addres',
 'source_port',
 'dest_ip_address',
 'dest_port',
 'total_packets_used',
 'bytes_flow',
 'network_flags',
 'bytes_per_packet',
 'packets_per_seconds',
 'hour_of_day',
 'is_common_port',
 'has_SYN',
 'has_ACK',
 'has_RST',
 'has_FIN',
 'is_hex_flag',
 'source_ip_freq',
 'dest_ip_freq',
 'class',
 'network_protocol_ICMP',
 'network_protocol_TCP',
 'network_protocol_UDP']

In [115]:
df = df[[
    'session_start_time',
    'duration',
    'source_ip_addres',
    'source_port',
    'dest_ip_address',
    'dest_port',
    'total_packets_used',
    'bytes_flow',
    'network_flags',
    'bytes_per_packet',
    'packets_per_seconds',
    'hour_of_day',
    'is_common_port',
    'has_SYN',
    'has_ACK',
    'has_RST',
    'has_FIN',
    'is_hex_flag',
    'source_ip_freq',
    'dest_ip_freq',
    'network_protocol_ICMP',
    'network_protocol_TCP',
    'network_protocol_UDP',
    'class'
]]

In [117]:
final_df = df[[
    "duration",
    'total_packets_used',
    'bytes_flow',
    'bytes_per_packet',
    'packets_per_seconds',
    'hour_of_day',
    'is_common_port',
    'has_SYN',
    'has_ACK',
    'has_RST',
    'has_FIN',
    'is_hex_flag',
    'source_ip_freq',
    'dest_ip_freq',
    'network_protocol_ICMP',
    'network_protocol_TCP',
    'network_protocol_UDP',
    'class'
]]

In [118]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 151332 entries, 18 to 172821
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   duration               151332 non-null  float64
 1   total_packets_used     151332 non-null  int64  
 2   bytes_flow             151332 non-null  float64
 3   bytes_per_packet       151332 non-null  float64
 4   packets_per_seconds    151332 non-null  float64
 5   hour_of_day            151332 non-null  int32  
 6   is_common_port         151332 non-null  int64  
 7   has_SYN                151332 non-null  int64  
 8   has_ACK                151332 non-null  int64  
 9   has_RST                151332 non-null  int64  
 10  has_FIN                151332 non-null  int64  
 11  is_hex_flag            151332 non-null  int64  
 12  source_ip_freq         151332 non-null  int64  
 13  dest_ip_freq           151332 non-null  int64  
 14  network_protocol_ICMP  151332 non-null  

In [ ]:
PATH_ON_BUCKET = "preprocessed/CIDDS-001.csv"
gcs.upload_to_bucket(final_df, PATH_ON_BUCKET)